In [160]:
include("util.jl");

## Day 1

In [2]:
depths = getinput(1) |> split .|> int

function count_ascending(arr)
    (d′ > d for (d, d′) in zip(arr, arr[2:end])) |> sum
end;

In [3]:
@time count_ascending(depths)

  0.095016 seconds (371.25 k allocations: 21.760 MiB, 15.82% gc time, 99.97% compilation time)


1532

In [4]:
sums3 = (depths[i:i+2] for i ∈ 1:(length(depths)-2)) .|> sum
@time count_ascending(sums3)

  0.000007 seconds (2 allocations: 15.766 KiB)


1571

## Day 2

In [5]:
dirs = Dict(
    "forward" => [1, 0],
    "down" => [0, 1],
    "up" => [0, -1]
)

function parse_row(row)
    dir, mag = split(row)
    return dirs[dir] * int(mag)
end

commands = split(getinput(2), "\n") .|> parse_row

pos = sum(commands)
pos, *(pos...)

([1832, 1172], 2147104)

In [6]:
pos = [0, 0]
aim = 0

for row in split(getinput(2), "\n")
    command, mag = split(row)
    if command == "forward"
       pos += [1, aim] * int(mag)
    else
        aim += (command == "down" ? 1 : -1) * int(mag)
    end    
end

pos, *(pos...)

([1832, 1116059], 2044620088)

## Day 3

In [7]:
rows = getinput(3) |> split .|> collect
mat = (hcat(rows...) .== '1')'
N = length(rows)
γ = mapslices(mat, dims=1) do col
    sum(col) > N // 2
end[:]
ϵ = γ .== 0

function base10(bitvec)
    str = convert.(Int, bitvec) |> join
    return parse(Int, str, base=2)
end

power = base10(γ) * base10(ϵ)

693486

In [10]:
function oxygen(mat, col)
    N = size(mat, 1)
    most_common = sum(mat[:,col]) / N >= 0.5
    idxs = filter(i -> mat[i, col] == most_common, 1:N)
    return mat[idxs,:]
end

m = mat
for i in 1:12
    m = oxygen(m, i)
end
base10(m[:])

933

In [15]:
function co2(mat, col)
    N = size(mat, 1)
    most_common = sum(mat[:,col]) / N >= 0.5
    idxs = filter(i -> mat[i, col] != most_common, 1:N)
    return mat[idxs,:]
end

m = mat
for i in 1:12
    if size(m, 1) == 1
        break;
    end
    m = co2(m, i)
end
base10(m[:])

3622

In [16]:
3622 * 933

3379326

## Day 4

In [26]:
numbers, boards... = split(getinput(4), "\n\n")

numbers = split(numbers, ",") .|> int

function parse_board(b)
    board = b |> split .|> int
    return reshape(board, 5, 5)
end
boards = boards .|> parse_board

board_hits = [zeros(Bool, 5, 5) for _ in 1:length(boards)]

completed_row(h) = any(mapslices(sum, h, dims=1).==5)
completed(h) = completed_row(h) || completed_row(h')


winner = nothing

for number in numbers
    for (board, hits) in zip(boards, board_hits)
        idx = findfirst(x -> x == number, board)
        if idx === nothing
            continue
        end
        hits[idx] = true
        if completed(hits)
            winner = board, hits, number
        end
    end

    if winner !== nothing
        break
    end
end

board, mask, number = winner
score = sum((mask .== 0) .* board) * number

69579

In [27]:
num = -1
lastwinner = -1
winners = []
board_hits = [zeros(Bool, 5, 5) for _ in 1:length(boards)]

for number in numbers
    for (b_idx, (board, hits)) in enumerate(zip(boards, board_hits))
        idx = findfirst(x -> x == number, board)
        if idx === nothing
            continue
        end
        hits[idx] = true
        if completed(hits) && b_idx ∉ winners
            push!(winners, b_idx)
            lastwinner = b_idx
        end
    end
    if length(winners) == length(boards)
        num = number
        break
    end
end

board, mask = boards[lastwinner], board_hits[lastwinner]
score = sum((mask .== 0) .* board) * num


14877

# Day 5

In [82]:
lines = getinput(5) |> spliton("\n")

coords = lines .|> spliton(" -> ")
to_coords(x) = int.(split(x, ',')) .+ 1
lines = reshape(vcat(coords...), 2, :) .|> to_coords

2×500 Matrix{Vector{Int64}}:
 [782, 722]  [335, 552]  [26, 154]   …  [43, 970]  [364, 689]  [646, 881]
 [782, 612]  [120, 552]  [766, 894]     [984, 29]  [364, 149]  [266, 501]

In [94]:
"""
    ordering(i, j)

1 if i < j
0 if i = j
-1 if i > j
"""
ordering(i, j) = i < j ? 1 : (i == j ? 0 : -1)

function displacement_unit_vector(a, b)
    x = ordering(a[1], b[1])
    y = ordering(a[2], b[2])
    return [x, y]
end

function solve(vec_allowed)
    grid = zeros(Int, 1000, 1000)
    for (pointa, pointb) in eachcol(lines)
        d = displacement_unit_vector(pointa, pointb)
        if !vec_allowed(d)
            continue
        end
        pointc = copy(pointa)
        while true
            grid[pointc...] += 1
            pointc != pointb || break
            pointc += d
        end
    end

    sum(grid .> 1)
end

solve(d -> sum(abs.(d)) == 1)

3990

In [95]:
solve(d -> true)

21305

# Day 6

In [107]:
starting = getinput(6) |> spliton(",") .|> int
starting .+= 1
ages = zeros(Int, 9)
for age in starting
    ages[age] += 1
end

function alive_after(starting_ages, days)
    ages = copy(starting_ages)
    for day in 1:days
        new = ages[1]
        ages = vcat(ages[2:end], [new])
        ages[7] += new
    end

    sum(ages)
end

alive_after(ages, 80)

395627

In [108]:
alive_after(ages, 256)

1767323539209

# Day 7

In [177]:
inp = 7 |> getinput |> spliton(",") .|> int
N = maximum(inp)+1
occupancies = zeros(Int, N)
for i in inp
    occupancies[i+1] += 1
end

function cost_for_distance(d, fuel_for_distance)
    distances = (1:N) .- d .|> abs
    fuels = distances .|> fuel_for_distance
    return sum(fuels .* occupancies)
end

function solve(fuel_for_distance)
    costs = 1:N .|> d -> cost_for_distance(d, fuel_for_distance)
    return costs |> minimum
end

@time solve(n -> n)

  0.181652 seconds (400.48 k allocations: 105.023 MiB, 10.74% gc time, 80.31% compilation time)


345197

In [185]:
@time cost_for_distance(sort(inp)[500], n->n)

  0.078429 seconds (144.21 k allocations: 8.908 MiB, 11.39% gc time, 98.99% compilation time)


345199

In [178]:
triangle(n) = convert(Int, n * (n+1) / 2)

@time solve(triangle)

  0.151090 seconds (310.88 k allocations: 99.077 MiB, 18.36% gc time, 76.31% compilation time)


96361606

In [179]:
# faster - nice proof that the minimum lies within x̄ ± 0.5
@time begin m = sum(inp) / length(inp)
    candidates = m ± 0.5 .|> round .|> int
    costs = candidates .|> x -> cost_for_distance(x, triangle)
    minimum(costs)
end

  0.076791 seconds (163.10 k allocations: 10.010 MiB, 98.78% compilation time)


96361606